In [1]:
import numpy as np
import scipy.io as sio 
import math
import matplotlib.pyplot as plt
from ntfmodel import *
from utils import *

In [2]:
datapath = '../../MATLAB/ComponentAnalysisExperiments/data/'
fn = 'h02-jasper'
Lr = 12
matdict = sio.loadmat(datapath + fn)
Y = matdict['hsiten']
Sgt = matdict['Sgt'] 
# R = 6
# Sgt = np.random.uniform(size=(Y.shape[2],R))

### L-inf Normalized on mode-2
Ynmax = np.max(Y)
Yninf = np.linalg.norm(Y, ord=np.inf, axis=2, keepdims=True)
Yn2 = np.linalg.norm(Y, ord=2, axis=2, keepdims=True)
Y=Y/Yninf

[I,J,K] = Y.shape
[K,R] = Sgt.shape
print(fn)
print(f'[I,J,K]=>[{I},{J},{K}]   [Lr,R]=>[{Lr},{R}]')
print()

h02-jasper
[I,J,K]=>[100,100,198]   [Lr,R]=>[12,4]



C:\Users\wilo_\anaconda3\envs\tf-gpu\lib\site-packages\scipy\io\matlab\mio.py:224: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [3]:
i=0
model = LrModel(Y,Lr,R,seed=i)
model.lrate = 0.001
model.MaxDelta = 1e-8
model.RegWeight = 0.0
model.AscWeight = 0.0
model.run_optimizer()

# Compute endmembers using spatial components
# and reconstructed tensor
AbundanceThreshold = 0.95
AbundanceFromTarget = False
Sprime = get_endmembers(model, 
    AbundanceThreshold, 
    fromtarget=AbundanceFromTarget)
(Sprime2,p) = reorder(Sprime,Sgt)
print(f'Reorder: {p}')
plot_decomposition(model,Sgt,Sprime2,p)
plt.show()


NameError: in user code:

    C:\Users\wilo_\OneDrive - University of Puerto Rico\tensorflow\NNTF\ntfmodel.py:113 cost  *
        tc = self.loss()             + self.reg_term() + self.asc_term()
    C:\Users\wilo_\OneDrive - University of Puerto Rico\tensorflow\NNTF\ntfmodel.py:95 loss  *
        se = tf.math.squared_difference(self.Y,self())
    C:\Users\wilo_\OneDrive - University of Puerto Rico\tensorflow\NNTF\ntfmodel.py:76 __call__  *
        op1 = self.Eop()
    C:\Users\wilo_\OneDrive - University of Puerto Rico\tensorflow\NNTF\ntfmodel.py:71 Eop  *
        self.B.assign(Bunit)

    NameError: name 'Bunit' is not defined


In [ ]:
# Compute Fully Constrained Least Squares 
A = fcls_np(Y,Sprime2)
Agt = read_agt(matdict)
nx = math.floor(matdict['nx'])
mtx = compute_metrics(i,Sgt, Sprime2, A, Agt)
plot_abundance(Agt,A,nx)
plt.show()

In [ ]:
[Yc,Ynorms,Enorms] = model.component_norms()
Yc = Yc[p,:,:,:]
Ynorms=Ynorms[p]
Enorms=Enorms[p]
ycn_sort = np.argsort(Enorms)[::-1]
print(Enorms)
print(ycn_sort,Ynorms,Enorms)

In [ ]:
plt.imshow(hsi2rgb(Y))
plt.show()

Y1 = Y-Yc[ycn_sort[0],:,:,:]
plt.imshow(hsi2rgb(Y1))
plt.show()

Y2 = Y1-Yc[ycn_sort[1],:,:,:]
Y2max = np.max(Y2)/2
Y2max
Y2 = np.where(Y2<Y2max,1e-15,Y2)
plt.imshow(hsi2rgb(Y2))
plt.show()

Y3 = Y2-Yc[ycn_sort[3],:,:,:]
plt.imshow(hsi2rgb(Y3))
plt.show()

Y4 = Y3-Yc[ycn_sort[2],:,:,:]
Y4 = Y4/np.linalg.norm(Y4, ord=np.inf, axis=2, keepdims=True)
plt.imshow(hsi2rgb(Y4))
plt.show()

In [ ]:

model3 = LrModel(Y2,Lr,2,seed=i)
model3.lrate = 0.001
model3.MaxDelta = 1e-8
model3.AscWeight = 0.0
model3.RegWeight = 0.0
model3.run_optimizer()

In [ ]:
plt.plot(model3.C.numpy().transpose())
plt.show()
A=model.A.numpy()
B=model.B.numpy()
C=model.C.numpy()
for n,j in enumerate(range(2,4)):
    A[ycn_sort[j],:,:] = model3.A[n,:,:]
    B[ycn_sort[j],:,:] = model3.B[n,:,:]
    C[ycn_sort[j],:] = model3.C[n,:]
model.A.assign(A)
model.B.assign(B)
model.C.assign(C);

plt.plot(model.C.numpy().transpose())
plt.show()


In [ ]:
# Compute endmembers using spatial components
# and reconstructed tensor
AbundanceThreshold = 0.95
AbundanceFromTarget = False
Sprime = get_endmembers(model, 
    AbundanceThreshold, 
    fromtarget=AbundanceFromTarget)
(Sprime,p) = reorder(Sprime,Sgt)
print(f'Reorder: {p}')
plot_decomposition(model,Sgt,Sprime,p)
plt.show()

# Compute Fully Constrained Least Squares 
A = fcls_np(Y,Sprime)
Agt = read_agt(matdict)
nx = math.floor(matdict['nx'])
mtx = compute_metrics(i,Sgt, Sprime, A, Agt)
plot_abundance(Agt,A,nx)
plt.show()